### Fine-tuning a LLM to text classification ###

In [18]:
import random

# 1. Load Positive Tweets
with open("twitter-datasets/train_pos_full.txt", "r", encoding="utf-8") as file:
    positive_tweets = file.readlines()
    #positive_tweets = positive_tweets[:len(positive_tweets)]  # Select only half of the positive tweets

# 2. Load Negative Tweets
with open("twitter-datasets/train_neg_full.txt", "r", encoding="utf-8") as file:
    negative_tweets = file.readlines()
    #negative_tweets = negative_tweets[:len(negative_tweets)//10]  # Select only half of the negative tweets

# 3. Create the texts and labels lists
texts = positive_tweets + negative_tweets
labels = [1] * len(positive_tweets) + [0] * len(negative_tweets)

# 4. Shuffle the dataset
data = list(zip(texts, labels))  # Combine texts and labels into tuples
random.shuffle(data)  # Shuffle the dataset

# 5. Unzip the shuffled data back into texts and labels
texts, labels = zip(*data)

# Convert back to list if you need to work with them as lists
texts = list(texts)
labels = list(labels)

# Check the first few entries to ensure everything looks correct

In [19]:
from transformers import AutoTokenizer

# Load a pretrained tokenizer
model_name = "cardiffnlp/twitter-roberta-base-sentiment"  # You can replace this with another model
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Tokenize the data
tokenized_data = tokenizer(
    texts,
    padding=True,
    truncation=True,
    max_length=128,
    return_tensors="pt"
)

# Inspect tokenized output
#print(tokenized_data)


In [20]:
import torch

class TweetDataset(torch.utils.data.Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]

        # Tokenize
        tokens = self.tokenizer(
            text,
            padding="max_length",
            truncation=True,
            max_length=self.max_length,
            return_tensors="pt"
        )
        return {
            "input_ids": tokens["input_ids"].squeeze(0),
            "attention_mask": tokens["attention_mask"].squeeze(0),
            "label": torch.tensor(label, dtype=torch.long)
        }

# Create dataset and dataloaders
dataset = TweetDataset(texts, labels, tokenizer)
train_loader = torch.utils.data.DataLoader(dataset, batch_size=16, shuffle=True)


In [22]:
from transformers import AutoModelForSequenceClassification

# Load model with classification head
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)


In [23]:
from transformers import TrainingArguments, Trainer
from datasets import Dataset

# Prepare the dataset for Trainer
hf_dataset = Dataset.from_dict({"text": texts, "label": labels})
tokenized_dataset = hf_dataset.map(
    lambda x: tokenizer(x["text"], padding="max_length", truncation=True, max_length=256),
    batched=True
)

# Split the dataset into training and evaluation sets



tokenized_dataset = tokenized_dataset.rename_column("label", "labels")
tokenized_dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels"])




Map:   0%|          | 0/2500000 [00:00<?, ? examples/s]

In [24]:
train_size = int(0.8 * len(tokenized_dataset))
eval_size = len(tokenized_dataset) - train_size

train_dataset, eval_dataset = torch.utils.data.random_split(tokenized_dataset, [train_size, eval_size])


In [25]:
import accelerate

# Train the model
# Initialize the Trainer
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
model = model.to(device)  # Move model to the MPS device

print("Using device:", device)

Using device: mps


In [27]:
torch.mps.empty_cache()

In [ ]:

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    evaluation_strategy="no",
    save_strategy="steps",  # Save checkpoints at regular step intervals
    save_steps=10000,       # Save every 10,000 steps
    use_mps_device=True     # Use MPS if available
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,  # Use a separate dataset for evaluation in real use
)

trainer.train()


  0%|          | 0/31250 [00:00<?, ?it/s]

RuntimeError: MPS backend out of memory (MPS allocated: 17.98 GB, other allocations: 142.95 MB, max allowed: 18.13 GB). Tried to allocate 192.00 MB on private pool. Use PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 to disable upper limit for memory allocations (may cause system failure).

In [13]:
trainer.save_model("./distilbert-base-uncased")
tokenizer.save_pretrained("./distilbert-base-uncased")

('./distilbert-base-uncased/tokenizer_config.json',
 './distilbert-base-uncased/special_tokens_map.json',
 './distilbert-base-uncased/vocab.txt',
 './distilbert-base-uncased/added_tokens.json',
 './distilbert-base-uncased/tokenizer.json')

In [14]:
torch.save({
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': trainer.optimizer.state_dict(),
    'scheduler_state_dict': trainer.lr_scheduler.state_dict(),
}, "./training_state.pth")


In [10]:
eval_results = trainer.evaluate()
print(eval_results)

  0%|          | 0/3907 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [15]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import numpy as np

model = AutoModelForSequenceClassification.from_pretrained("./distilbert-base-uncased")
tokenizer = AutoTokenizer.from_pretrained("./distilbert-base-uncased")

data_path = "data/twitter-datasets/"
test_path = f"{data_path}test_data.txt"


with open(test_path, "r") as f:
    test_tweets = [line.strip() for line in f]


In [ ]:
from tqdm import tqdm
labels = []

# Analyze sentiment
for tweet in tqdm(test_tweets):
    inputs = tokenizer(tweet, return_tensors="pt", truncation=True, max_length=512)
    outputs = model(**inputs)
    probs = torch.nn.functional.softmax(outputs.logits, dim=-1)
    negative_prob, neutral_prob,positive_prob = probs[0].tolist()
    if positive_prob > negative_prob:
        label = 1  # Positive
    else:
        label = -1  # Negative
    labels.append(label)

ids = np.arange(1, len(labels) + 1)

100%|██████████| 10000/10000 [02:48<00:00, 59.33it/s]


In [17]:
from helpers import create_csv_submission

create_csv_submission(ids, labels, "data/submission_60_perce_fine_tuned_distilbert-base-uncased.csv")


### Before fine-tuning ###

In [15]:
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased")
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

data_path = "data/twitter-datasets/"
test_path = f"{data_path}test_data.txt"


with open(test_path, "r") as f:
    test_tweets = [line.strip() for line in f]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
from tqdm import tqdm
labels = []

# Analyze sentiment
for tweet in tqdm(test_tweets):
    inputs = tokenizer(tweet, return_tensors="pt", truncation=True, max_length=512)
    outputs = model(**inputs)
    probs = torch.nn.functional.softmax(outputs.logits, dim=-1)
    negative_prob, positive_prob = probs[0].tolist()
    if positive_prob > negative_prob:
        label = 1  # Positive
    else:
        label = -1  # Negative
    labels.append(label)

ids = np.arange(1, len(labels) + 1)

In [22]:
print(labels)

[-1, -1, -1, -1, 1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1

In [17]:
from helpers import create_csv_submission

create_csv_submission(ids, labels, "data/submission_pre_trained_distilbert-base-uncased.csv")
